In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from models.detector import face_detector
from models.detector.iris_detector import IrisDetector
import sys
import warnings
warnings.filterwarnings("ignore")

C:\Users\jason\anaconda3\envs\yolov4\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jason\anaconda3\envs\yolov4\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\jason\anaconda3\envs\yolov4\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\jason\anaconda3\envs\yolov4\lib\site-packages\tensorflow\pyth

In [2]:
fd = face_detector.FaceAlignmentDetector(
    lmd_weights_path="./models/detector/FAN/2DFAN-4_keras.h5",
     fd_type="s3fd"
    # 2DFAN-4_keras.h5, 2DFAN-1_keras.h5
)

Instructions for updating:
Colocations handled automatically by placer.
loading S3FD model............
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, None, None, 6 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv1_2 (Conv2D)                (None, None, None, 6 36928       conv1_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, None, None, 6 0           conv1_2[0][0]                    
______

In [3]:
def resize_image(im, max_size=768):
    if np.max(im.shape) > max_size:
        ratio = max_size / np.max(im.shape)
        print(f"Resize image to ({str(int(im.shape[1]*ratio))}, {str(int(im.shape[0]*ratio))}).")
        return cv2.resize(im, (0,0), fx=ratio, fy=ratio)
    return im


# Test images are obtained on https://www.pexels.com/
im = cv2.imread(r"C:\Users\jason\Facial-Recognition-FaceNet-master\database\id\eric.jpg")[..., ::-1]
im = resize_image(im) # Resize image to prevent GPU OOM.
bboxes, landmarks = fd.detect_face(im, with_landmarks=True)

Resize image to (768, 432).
Building FAN for landmarks detection...
loading FAN model............
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input0 (InputLayer)             (None, 3, 256, 256)  0                                            
__________________________________________________________________________________________________
conv10.23005182903749222_pad (Z (None, 3, 262, 262)  0           input0[0][0]                     
__________________________________________________________________________________________________
conv10.23005182903749222 (Conv2 (None, 64, 128, 128) 9472        conv10.23005182903749222_pad[0][0
__________________________________________________________________________________________________
bn10.20053864458101422 (BatchNo (None, 64, 128, 128) 256         conv10.23005182903749222[0][0]   
___________

m2.b3_4.bn30.9750356847559866 ( (None, 64, 32, 32)   256         m2.b3_4.conv20.16483593974391375[
__________________________________________________________________________________________________
m2.b1_4.bn30.19355388224651415  (None, 64, 64, 64)   256         m2.b1_4.conv20.5932910761721709[0
__________________________________________________________________________________________________
m2.b3_40.5596414327425956 (Acti (None, 64, 32, 32)   0           m2.b3_4.bn30.9750356847559866[0][
__________________________________________________________________________________________________
m2.b1_40.4442770372311482 (Acti (None, 64, 64, 64)   0           m2.b1_4.bn30.19355388224651415[0]
__________________________________________________________________________________________________
m2.b3_4.conv30.8155829984460058 (None, 64, 34, 34)   0           m2.b3_40.5596414327425956[0][0]  
__________________________________________________________________________________________________
m2.b1_4.co

m3.b2_1.bn30.2871328456186292 ( (None, 64, 4, 4)     256         m3.b2_1.conv20.00992581586688901[
__________________________________________________________________________________________________
m3.b2_10.132177635875386 (Activ (None, 64, 4, 4)     0           m3.b2_1.bn30.2871328456186292[0][
__________________________________________________________________________________________________
m3.b2_1.conv30.6461212950711416 (None, 64, 6, 6)     0           m3.b2_10.132177635875386[0][0]   
__________________________________________________________________________________________________
m3.b2_1.conv30.6461212950711416 (None, 64, 4, 4)     36864       m3.b2_1.conv30.6461212950711416_p
__________________________________________________________________________________________________
m3.b2_10.965103653090775 (Conca (None, 256, 4, 4)    0           m3.b2_1.conv10.3921586005815776[0
                                                                 m3.b2_1.conv20.00992581586688901[
          

In [5]:
import numpy as np
import cv2



video_size = (640,480)
space = 25
shrink_frame_scale=1
recover_size = lambda x: x*(int(1/shrink_frame_scale))

webcam = cv2.VideoCapture(0)
if not webcam.isOpened():
    sys.exit("Could not open webcam, please open the camera!")   
    


fourcc = cv2.VideoWriter_fourcc(*'MJPG')
width = int(webcam.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(webcam.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("width:",width, "height:", height)
out = cv2.VideoWriter('output2.avi', fourcc, 20.0, video_size)


while True:
    
    # read frame from webcam 
    status, frame = webcam.read()     
    frame_size = frame.shape
    out.write(frame)
    
    small_frame = cv2.resize(frame, (0, 0), fx=shrink_frame_scale, fy=shrink_frame_scale)

    h, w, _ = frame.shape
    
    bboxes = fd.detect_face(small_frame, with_landmarks=False)
    
    number0 = len(bboxes)
    number = len(bboxes[0])
    
    print(bboxes)
    
    if number!=0:
   
        for i in range(number0):
            
             # Display detected face
            x0, y0, x1, y1, score = bboxes[i] 
            x0, y0, x1, y1 = map(int, [x0, y0, x1, y1])

            crop_face = small_frame[x0-space:x1+space, y0-space:y1+space, :]
            
            face = cv2.resize(crop_face,video_size)
            
             
            
            l = x1-x0
            w = y1-y0

            x0, y0, x1, y1 = map(recover_size,  [x0, y0, x1, y1])
            print(x0,y0,x1,y1)
            cv2.rectangle(frame, (y0,x0), (y1,x1), (0,255,0), 2)
  
    else:
        print("No face detected.")
        
        
        
       
    cv2.namedWindow("demo", cv2.WINDOW_NORMAL)
    cv2.imshow("demo", crop_face)
    
    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# release resources
webcam.release()
out.release()
cv2.destroyAllWindows()   

width: 640 height: 480
[array([174.3359832763672, 221.92475128173828, 436.4552459716797,
       415.05545806884766, array([1.], dtype=float32)], dtype=object)]
174 221 436 415
[array([173.74063110351562, 221.24224853515625, 436.8741455078125,
       414.5350646972656, array([1.], dtype=float32)], dtype=object)]
173 221 436 414
[array([174.5187873840332, 220.88877487182617, 434.2383003234863,
       413.95376205444336, array([1.], dtype=float32)], dtype=object)]
174 220 434 413
[array([168.40694618225098, 220.77365112304688, 433.8067569732666,
       415.2040710449219, array([1.], dtype=float32)], dtype=object)]
168 220 433 415
[array([171.43231201171875, 221.64683151245117, 435.29437255859375,
       414.82021713256836, array([1.], dtype=float32)], dtype=object)]
171 221 435 414
[array([167.4515142440796, 220.49722290039062, 433.95038509368896,
       416.15199279785156, array([1.], dtype=float32)], dtype=object)]
167 220 433 416
[array([172.99822807312012, 221.40933990478516, 435.6838

175 221 433 411
[array([177.12420082092285, 221.65818214416504, 436.3540287017822,
       414.2609348297119, array([1.], dtype=float32)], dtype=object)]
177 221 436 414
[array([174.75008010864258, 221.26712608337402, 434.79118728637695,
       412.17967796325684, array([1.], dtype=float32)], dtype=object)]
174 221 434 412
[array([176.91218376159668, 222.82009315490723, 434.90360832214355,
       411.9092044830322, array([1.], dtype=float32)], dtype=object)]
176 222 434 411
[array([170.05930042266846, 223.47021865844727, 433.11139392852783,
       412.34701919555664, array([1.], dtype=float32)], dtype=object)]
170 223 433 412
[array([176.22479057312012, 222.46936416625977, 432.0940532684326,
       412.882999420166, array([0.9999995], dtype=float32)], dtype=object)]
176 222 432 412
[array([173.96948528289795, 221.2688980102539, 430.3810758590698,
       412.36278533935547, array([1.], dtype=float32)], dtype=object)]
173 221 430 412
[array([175.08499908447266, 224.98927879333496, 435.690

177 222 433 412
[array([176.7163791656494, 223.24543952941895, 434.8172092437744,
       413.2883930206299, array([1.], dtype=float32)], dtype=object)]
176 223 434 413
[array([173.019118309021, 221.30651473999023, 432.77946376800537,
       413.0939140319824, array([1.], dtype=float32)], dtype=object)]
173 221 432 413
[array([178.33768272399902, 223.58770179748535, 436.654333114624,
       414.14190101623535, array([1.], dtype=float32)], dtype=object)]
178 223 436 414
[array([175.6095027923584, 223.09863090515137, 436.21942710876465,
       412.2261486053467, array([1.], dtype=float32)], dtype=object)]
175 223 436 412
[array([175.52700996398926, 224.36169815063477, 435.17581367492676,
       414.681583404541, array([1.], dtype=float32)], dtype=object)]
175 224 435 414
[array([176.76359367370605, 222.33298110961914, 434.3902416229248,
       412.4381446838379, array([1.], dtype=float32)], dtype=object)]
176 222 434 412
[array([174.59765434265137, 222.92190170288086, 437.07272148132324,


174 219 435 413
[array([175.67044067382812, 219.9571533203125, 435.080322265625,
       412.4413757324219, array([1.], dtype=float32)], dtype=object)]
175 219 435 412
[array([177.57648086547852, 220.31536102294922, 433.9001808166504,
       412.9889450073242, array([1.], dtype=float32)], dtype=object)]
177 220 433 412
[array([177.2780818939209, 220.52515029907227, 434.26703453063965,
       413.8796730041504, array([1.], dtype=float32)], dtype=object)]
177 220 434 413
[array([177.8474521636963, 221.60197830200195, 436.94642066955566,
       413.9438362121582, array([1.], dtype=float32)], dtype=object)]
177 221 436 413
[array([174.12928581237793, 220.53746795654297, 437.38673210144043,
       414.0960159301758, array([1.], dtype=float32)], dtype=object)]
174 220 437 414
[array([177.96603202819824, 223.96709823608398, 436.9367961883545,
       412.6141777038574, array([0.9999995], dtype=float32)], dtype=object)]
177 223 436 412
[array([176.6214485168457, 221.65722465515137, 437.910236358

177 219 436 413
[array([179.22267532348633, 223.61025047302246, 438.7124214172363,
       415.7117977142334, array([1.], dtype=float32)], dtype=object)]
179 223 438 415
[array([179.60516738891602, 223.4255199432373, 438.36023330688477,
       415.7293529510498, array([1.], dtype=float32)], dtype=object)]
179 223 438 415
[array([178.6928310394287, 223.07657623291016, 438.20836448669434,
       415.28804779052734, array([1.], dtype=float32)], dtype=object)]
178 223 438 415
[array([180.23686027526855, 222.6364631652832, 435.8677349090576,
       413.5539131164551, array([0.9999995], dtype=float32)], dtype=object)]
180 222 435 413
[array([174.82071495056152, 222.4922924041748, 437.74164390563965,
       414.3206310272217, array([1.], dtype=float32)], dtype=object)]
174 222 437 414
[array([176.5877285003662, 220.57726287841797, 434.5742702484131,
       414.20304107666016, array([1.], dtype=float32)], dtype=object)]
176 220 434 414
[array([177.52054595947266, 221.83097648620605, 435.7614212

178 223 435 413
[array([176.39006423950195, 222.7872772216797, 435.5381660461426,
       413.09400939941406, array([1.], dtype=float32)], dtype=object)]
176 222 435 413
[array([176.87672233581543, 221.5629997253418, 435.69312858581543,
       413.33633041381836, array([1.], dtype=float32)], dtype=object)]
176 221 435 413
[array([176.40131187438965, 223.20566368103027, 436.0916500091553,
       414.1532802581787, array([1.], dtype=float32)], dtype=object)]
176 223 436 414
[array([173.9773406982422, 222.9439296722412, 435.9226837158203,
       412.6106472015381, array([1.], dtype=float32)], dtype=object)]
173 222 435 412
[array([175.21591567993164, 220.61199569702148, 433.2197914123535,
       413.37279891967773, array([1.], dtype=float32)], dtype=object)]
175 220 433 413
[array([175.1439151763916, 222.90317344665527, 436.6085453033447,
       412.348669052124, array([1.], dtype=float32)], dtype=object)]
175 222 436 412
[array([178.8169288635254, 222.33270835876465, 436.94821548461914,
 